In [ ]:
from llama_cpp import Llama

In [ ]:
QWEN_MODEL_PATH = r"model\qwen2.5-1.5b-instruct-q8_0.gguf"
LLAMA_MODEL_PATH = r"model\llama-2-7b-chat.Q4_K_M.gguf"
BIAS = 0.6

In [ ]:
qwen = Llama(
    model_path=QWEN_MODEL_PATH,
    n_ctx=4096,
    n_threads=8,   # adjust to your CPU
    verbose=False
)


In [ ]:
llama = Llama(
    model_path=LLAMA_MODEL_PATH,
    n_ctx=4096,
    n_threads=8,
    verbose=False
)

In [ ]:
def run_model(llm, prompt, max_tokens=512, temp=0.3):
    output = llm(
        prompt,
        max_tokens=max_tokens,
        temperature=temp,
        stop=["</s>"]
    )
    return output["choices"][0]["text"].strip()


In [ ]:
def extract_key_points(text):
    prompt = f"""
You are an expert teacher.

Task:
Extract only important exam-relevant points from the text.
Use ONLY the given text.
Do NOT add explanations.

Return as a numbered list.

Text:
\"\"\"
{text}
\"\"\"
"""
    return run_model(qwen, prompt, max_tokens=400, temp=0.2)

In [ ]:
def generate_questions_mcq(points,amount):
    prompt = f"""
SYSTEM:
You are an exam paper generator.
You MUST generate questions.
You MUST NOT ask the user to do anything.

TASK:
Generate exactly:
1. {amount} MCQs


RULES:
- Use ONLY the points below
- Do NOT add new information
- Do NOT explain
- Do NOT ask questions
- Output questions directly

POINTS:
{points}

OUTPUT:
"""
    return run_model(llama, prompt, max_tokens=600, temp=0.25)

In [ ]:
def generate_questions_short(points,amount):
    prompt = f"""
SYSTEM:
You are an exam paper generator.
You MUST generate questions.
You MUST NOT ask the user to do anything.

TASK:
Generate exactly:
1. {amount} short


RULES:
- Use ONLY the points below
- Do NOT add new information
- Do NOT explain
- Do NOT ask questions
- Output questions directly
- use _ in the end of question

POINTS:
{points}

OUTPUT:
"""
    return run_model(qwen, prompt, max_tokens=600, temp=0.25)

In [ ]:
def generate_questions_long(points,amount):
    prompt = f"""
SYSTEM:
You are an exam paper generator.
You MUST generate questions.
You MUST NOT ask the user to do anything.

TASK:
Generate exactly:
1. {amount} long


RULES:
- Use ONLY the points below
- Do NOT add new information
- Do NOT explain
- Do NOT ask questions
- Output questions directly

POINTS:
{points}

OUTPUT:
"""
    return run_model(llama, prompt, max_tokens=600, temp=0.25)

In [ ]:
def generate_questions_duplicates(questions):
    prompt = f"""
SYSTEM:
You are an expert exam question paper generator.

TASK:
Rewrite each question below into **TWO different versions** such that:
- The **meaning and answer remain exactly the same**
- The **wording and structure are different**
- The questions sound **natural, clear, and exam-appropriate**

RULES:
- Use ONLY the provided questions
- Do NOT add new information
- Do NOT change difficulty level
- Do NOT explain anything
- Do NOT ask anything
- Do NOT include numbering or labels like (Q1, Copy A, Copy B)
- Output ONLY the rewritten questions

FORMAT:
For each input question, output:
1) First reworded version
2) Second reworded version

QUESTIONS:
{questions}

OUTPUT:
"""
    return run_model(qwen, prompt, max_tokens=700, temp=0.25)


In [ ]:
def generate_exam_questions(text,mcq=True,short=False,long=False):
    key_points = extract_key_points(text)

    length = len(key_points.split('\n'))
    amount = round(length*BIAS)
    questions = {}
    if mcq:
        questions['mcq'] = generate_questions_mcq(key_points,amount)
    if short:
        questions['short'] = generate_questions_short(key_points,amount)
    if long:
        questions['long'] = generate_questions_long(key_points,amount)

    dupliates = generate_questions_duplicates(questions['short'])

    return {
        "key_points": key_points,
        "questions": questions,
        "dup":dupliates
    }

In [ ]:
sample_text = """
Operating systems act as an interface between computer hardware and the user.
They are responsible for managing hardware resources efficiently and ensuring
that application programs execute correctly. The primary resources managed by
an operating system include the CPU, main memory, secondary storage, and input/output devices.

The CPU management function of an operating system involves process scheduling.
Processes are programs in execution, and the operating system decides which
process runs on the CPU and for how long. Scheduling algorithms such as First Come
First Served (FCFS), Shortest Job First (SJF), Priority Scheduling, and Round Robin
are used to achieve fairness and efficiency. Context switching allows the CPU to
switch from one process to another, enabling multitasking.

Memory management is another critical responsibility of an operating system.
The operating system keeps track of which parts of memory are in use and which
are free. It allocates memory to processes when required and deallocates it
when the process terminates. Techniques such as paging and segmentation are
used to manage memory efficiently. Virtual memory allows programs to execute
even if they are not fully loaded into physical memory.

File system management enables the operating system to organize data on storage
devices. Files are logical units of information, and directories are used to
group related files. The operating system provides operations such as file
creation, deletion, reading, writing, and permission management. Different file
allocation methods like contiguous, linked, and indexed allocation are used
to store files on disk.

Input/output management handles communication between the system and external
devices. Device drivers act as intermediaries between the operating system and
hardware devices. Buffering, caching, and spooling techniques are used to
improve I/O performance and efficiency.

The kernel is the core component of an operating system. It runs in privileged
mode and has complete access to system resources. The kernel is responsible
for process management, memory management, device management, and system calls.
System calls provide an interface through which user programs can request
services from the operating system.

Operating systems can be classified into different types based on their usage.
Batch operating systems execute jobs in batches without user interaction.
Time-sharing operating systems allow multiple users to interact with the system
simultaneously. Real-time operating systems are designed to respond to inputs
within strict time constraints. Distributed operating systems manage a group
of independent computers and make them appear as a single system.

Security and protection are essential aspects of operating system design.
The operating system ensures that unauthorized users do not access system
resources. Authentication, authorization, and access control mechanisms are
used to protect data and processes. Protection mechanisms prevent one process
from interfering with another process.

Modern operating systems also support virtualization, which allows multiple
operating systems to run on a single physical machine. Virtual machines are
managed by a hypervisor, which allocates hardware resources to each virtual
environment. Virtualization improves resource utilization and system isolation.

"""


In [ ]:
result = generate_exam_questions(sample_text,mcq=False,short=True,long=False)


In [ ]:
print("\n=== KEY POINTS ===\n")
print(result["key_points"])


In [ ]:
from collections import defaultdict

def isInt(value):
    try:
        int(value)
        return True
    except Exception as e:
        return False


def formate_mcq(q):
    data = q.split('\n')
    mcq_data=defaultdict(list)
    current_q = None
    for line in data:
        if line:
            if isInt(line[0]):
                current_q =  line.split(' ',1)[-1]
            elif line[0] in ['A','B','C','D']:
                mcq_data[current_q].append(line.split(' ',1)[-1])
    return mcq_data

def formate_short_and_long(q):
    data = q.split('\n')
    return [line.split(' ',1)[-1]for line in data if line and isInt(line[0])]



In [ ]:
print("\n=== QUESTIONS ===\n")
print(result['questions']['short'])


In [ ]:
formate_short_and_long(result["questions"]['short'])

In [ ]:
print("\n=== QUESTIONS ===\n")
print(result['dup'])

In [ ]:
def formate_short(data):
    data = result["questions"]['short'].split('\n')
    return [line.split(' ',1)[-1]for line in data if line and isInt(line[0])]


In [ ]:
mcqs = formate_mcq(result['questions']['mcq'])